In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import nltk
from seqeval.metrics import classification_report

## Named Entity Recognition (NER)

[NER](https://en.wikipedia.org/wiki/Named-entity_recognition) is a Natural Language Processing (NLP) task that consists in 
  identifying and classifying names of entities (people, location, companies, among others) in a given sentence.
For instance, given the following sentence:

```
Jim bought 300 shares of Acme Corp. in 2006.
```

a NER system should identify and classify the three named entities as in the following:

<font color='blue'>[Jim]<sub>Person</sub></font> bought 
300 shares of 
<font color='red'>[Acme Corp.]<sub>Organization</sub></font> in 
<font color='green'>[2006]<sub>Time</sub></font>.




### IOB Tagging Format
In order to cast NER as a sequence tagging problem, we usually make use of the [IOB Tagging Scheme](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)).
In this scheme, one *IOB tag* is attributed to each token (a token is a word or a punctuation mark) in the input sentence.

Jim | bought | 300 | shares | of | Acme | Corp. | in | 2006 | .
---|---|---|---|---|---|---|---|---|---
`I-PER` | `O` | `O` | `O` | `O` | `I-ORG` | `I-ORG` | `O` | `I-TIME` | `O`

As you can see, each tag identifies the type of the corresponding entity.
The tags `I-<type>` indicates that a token is *inside* an entity of type `<type>`.
Additionally, there is the *out* tag `O` indicating that the token is not part of any entity.
Usually, consecutive tokens classified with the same type are considered part of the same entity.
When there are two consecutive tokens that are part of two different entities but of the same type, we need to use a *begin* tag `B-<type>`. There is one `B-<type>` tag for each entity type.
Such tags indicate the first token of an entity
  when the previous token is part of another entity of the same type.
See the [Wikipedia article](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)) for more details.


### CoNLL-2003 Dataset

The CoNLL-2003 dataset is one of the most used datasets for NER.
In the following, we show a snippet of this dataset.

```
-DOCSTART- -X- -X- O

EU NNP I-NP I-ORG
rejects VBZ I-VP O
German JJ I-NP I-MISC
call NN I-NP O
to TO I-VP O
boycott VB I-VP O
British JJ I-NP I-MISC
lamb NN I-NP O
. . O O

Peter NNP I-NP I-PER
Blackburn NNP I-NP I-PER

(...)

-DOCSTART- -X- -X- O

Rare NNP I-NP O
Hendrix NNP I-NP I-PER
song NN I-NP O

(...)

```

Each line contains a token.
An empty line is used to separate sentences.
A line containing `-DOCSTART- -X- -X- O` is used to separate documents (for NER, usually, we disregard these document boundaries).
Each token contains four features separated by space:
```
word pos_tag chunk_tag ner_tag
```
The first feature `word` contains the string of the token (word or punctuation mark).
The second feature `pos_tag` indicates the part-of-speech of the token.
The third feature `chunk_tag` encodes chunking information (phrases).
And, finally, the last feature `ner_tag` comprises the NER annotations using IOB tagging scheme.

### Training and Evaluation Data
The CoNLL-2003 dataset is split into three parts: train, testa and testb.
The `train` split is obviously used to model training and the remaining two splits are used for model selection and testing.
We will use the `testa` split to evaluate our models.

In [ ]:
# # download train and dev files
# !wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.train
# !wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testa

## Encoder

Define a simple class to encode symbols (states and observations names/strings) as integers (IDs).
Basically, we use two entangled data structures: a dictionary (symbol -> ID) and a list (Id -> symbol).

Then, we can represent the HMM parameters as arrays and matrices indexed by integers (IDs) instead of strings.
But, at the same time, we can keep the mapping between symbol name (string) and its ID (integer).

In [ ]:
class Encoder:
    """Encode a set of N symbols (strings) into integer IDs from {0, 1, ..., N-1}

    The encoder can be frozen at some point and then no new symbol can be added.
    When a frozen encoder gets a request for an unknown symbol, it raises an
    exception.
    """

    def __init__(self, symbols: list = None, frozen: bool = False):
        """Create a new encoder with optional list of symbols.

        Args:
            symbols (list[str]): list of symbols (default is None)
            frozen (bool): whether the encoder includes new symbols passed to method `get_id(s)` (default is False)
        """
        self.symbol_to_id: dict[str, int] = {}
        self.id_to_symbol: list[str] = []
        self.frozen: bool = False

        if symbols is not None:
            for s in symbols:
                self.get_id(s)

        if frozen:
            self.frozen = True

    def get_id(self, s: str) -> int:
        """return the ID (integer) corresponding to symbols `s`.

        If symbols `s` has not been encoded already and `self.frozen == False`,
        then include `s` in the mapping, assign a new ID for it, and return it.
        If symbols `s` has not been encoded already and `self.frozen == True`,
        raise an exception.

        Args:
            s (str): symbol to be encoded

        Returns:
            int: ID of the encoded symbol
        """
        if s not in self.symbol_to_id:
            if self.frozen:
                raise ValueError(f"Symbol {s} not in frozen encoder {self}")

            new_id = len(self.id_to_symbol)
            self.symbol_to_id[s] = new_id
            self.id_to_symbol.append(s)
            return new_id

        return self.symbol_to_id[s]

    def get_symbol(self, id: int) -> str:
        """return the symbols associated with `id`

        Args:
            id (int): ID of the symbol

        Returns:
            str: the symbol associated with the given ID
        """
        return self.id_to_symbol[id]

    def __repr__(self):
        return f"Encoder: {self.symbol_to_id}"

    def __len__(self):
        return len(self.id_to_symbol)

In [ ]:
# define the set of observations (x_enc) and the set of states (y_enc)
x_enc = Encoder()
y_enc = Encoder()

### Encode a token
In the following function, we can apply some transformations to the word string in order to make the taggin problem easier.
One issue with HMM for POS tagging is that there are so many different words, 
  and we need to convert each word into an ID (integer).
Specially, there are related words like `car`, `Car` and `cars`, that should almost always get the same tag, 
  but can be encoded as different IDs.

In [ ]:
def encode_token(word, tag):
    """Get features of a token and return a pair (x,y)"""
    # we can apply any transformation to the word in order to make the problem easier
    # word = word.lower()  # lower case
    # word = word[:6]  # ignore long-word ending

    # encode word
    x = x_enc.get_id(word)
    # encode tag
    y = y_enc.get_id(tag)

    return x, y

The function below iterate over a CoNLL file (train or evaluation), 
  encode each word and its NER tag, 
  and return two lists: `xs` and `ys`.
Each element in `xs` comprises of a sentence of the file (np.array of encoded words).
And each element in `ys` comprises the corresponding sequence of tags (np.array of encoded tags).

In [ ]:
def load_examples(file):
    xs = []
    ys = []
    with open(file) as f:
        x_sent = []
        y_sent = []
        for line in f:
            if len(line.strip()) == 0:
                # a blank line separates sentences
                if len(x_sent) > 0:
                    # end of a sentence
                    xs.append(np.array(x_sent))
                    ys.append(np.array(y_sent))
                    x_sent = []
                    y_sent = []
            elif line.startswith("-DOCSTART-"):
                # start of a new document (ignore document limits)
                pass
            else:
                # a token contains four fields (separated by space):
                # word, POS tag, noun-phrase tag, and NER tag
                # the function `encode_example(...)` gets the word and NER tag
                # of a token and return an encoded pair (x,y)
                word, _, _, tag = line.split()
                x, y = encode_token(word, tag)
                x_sent.append(x)
                y_sent.append(y)

    return xs, ys

Load train and evaluation (dev) datasets.

In [ ]:
x_train, y_train = load_examples("eng.train")
x_dev, y_dev = load_examples("eng.testa")

Freeze the encoders to avoid bugs.

In [ ]:
x_enc.frozen = True
y_enc.frozen = True

Number of sentences in each dataset.

In [ ]:
print(f"# train exs: {len(x_train)}")
print(f"# dev exs: {len(x_dev)}")

Number of unique words in the datasets.

In [ ]:
print(f"# unique x: {len(x_enc)}")

Number of tags.

In [ ]:
print(f"# unique y: {len(y_enc)}")

Visualize the tags and their corresponding IDs.

In [ ]:
y_enc

Distribution of the tags.
We can already see an issue with NER: the output tag `O` is much more frequent than all other tags.

In [ ]:
_ = plt.hist(
    [y_enc.get_symbol(y) for y_seq in y_train for y in y_seq],
    bins=range(len(y_enc) + 1),
    align="left",
    density=True,
)
_ = plt.xticks(rotation=45)

So let us have a look at the distribution of tags ignoring the `O` tag.
Now, we have another issue because the `B-<type>` tags are also very rare when compared to the `I-<type>` tags.

In [ ]:
_ = plt.hist(
    [y_enc.get_symbol(y) for y_seq in y_train for y in y_seq if y != y_enc.get_id("O")],
    bins=range(len(y_enc) + 1),
    align="left",
    density=True,
)
_ = plt.xticks(rotation=45)

In [ ]:
from collections import Counter

counter = Counter([y_enc.get_symbol(y) for y_seq in y_train for y in y_seq])
total = sum(counter.values())
print({tag: count / total for tag, count in counter.items()})

## Evaluation Script
NER models are usually evaluated using the CoNLL-2003 evaluation metric.
This metric considers [precision, recall and f1-score](https://en.wikipedia.org/wiki/Precision_and_recall).
In this implementation, these values are computed with respect to entities instead of individual tokens.
A predicted entity is considered correct only if the exact tokens are correctly identified and classified.

We use the seqeval library to compute the CoNLL metric.
More specifically, we use the `classification_report` function 
  which reports precision, recall, f1-score and support for each entity type plus three averages over the entity types: micro, macro and weighted.
In the following, you can see an example of this classification report.
The most important value in this table is the micro-averaged f1-score (`0.55` in the example).

```
              precision    recall  f1-score   support

         LOC       0.84      0.78      0.81      1837
        MISC       0.74      0.70      0.72       922
         ORG       0.21      0.76      0.32      1341
         PER       0.69      0.54      0.60      1842

   micro avg       0.45      0.69      0.55      5942
   macro avg       0.62      0.70      0.61      5942
weighted avg       0.63      0.69      0.62      5942
```

In [ ]:
def ids_to_tags(y):
    y_tags = []
    for y_k in y:
        y_tags.append([y_enc.get_symbol(tag) for tag in y_k])
    return y_tags


def cr_eval(y, y_pred):
    y = ids_to_tags(y)
    y_pred = ids_to_tags(y_pred)
    return classification_report(y, y_pred)

## Baseline method: Most Frequent Tag for each Token
It is always a good idea to implement a simple baseline method in order to have a reference performance.
In the following, we evaluate a baseline that, for each token, predicts the most frequent tag for this token in the training data.

In [ ]:
# count occurrences of each word (row) x tag (column) on training data
freq_x_y = np.zeros((len(x_enc), len(y_enc)))
for x_k, y_k in zip(x_train, y_train):
    for x_t, y_t in zip(x_k, y_k):
        freq_x_y[x_t, y_t] += 1

# most frequent tag for each word
most_freq_x = freq_x_y.argmax(axis=1)

pred_y = []
for x_k in x_dev:
    pred_y.append(np.array([most_freq_x[x_t] for x_t in x_k]))

print(f'Evaluation of "Most Frequent per Token":\n\n{cr_eval(y_dev, pred_y)}')

#### You can continue here with your answer + code (or program outside the notebook, both is fine).